In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import itertools
import torch.utils.data as data_utils

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
data_transforms = transforms.Compose([     
                        transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225] )
    ])

In [4]:
batch_size = 10

#ТРЕЙНИНГ
train_dataset = torchvision.datasets.ImageFolder(root='./animals/train', 
                                                 transform=data_transforms)
#СЛАЙСИНГ (через random_split)
len_sliced = 2000
tr_sliced = torch.utils.data.random_split(train_dataset, [len_sliced, len(train_dataset)-len_sliced])[0]

train_loader = torch.utils.data.DataLoader(tr_sliced, batch_size=batch_size, 
                                    shuffle=True,  num_workers=2)


#ТЕСТ
test_dataset = torchvision.datasets.ImageFolder(root='./animals/val',
                                             transform=data_transforms)
len_sliced = 1000
test_sliced = torch.utils.data.random_split(test_dataset, [len_sliced, len(test_dataset)-len_sliced])[0]

test_loader  = torch.utils.data.DataLoader(test_sliced, batch_size=batch_size, 
                                    shuffle=True, num_workers=2) 



In [5]:
print(train_dataset.classes)
print(test_dataset.classes)

['cat', 'dog', 'wild']
['cat', 'dog', 'wild']


In [6]:
# Сохранение названия классов
class_names = train_dataset.classes

In [2]:
inputs, classes = next(iter(train_loader))

inputs.shape

classes

img = torchvision.utils.make_grid(inputs, nrow = 3)
img = img.numpy().transpose((1, 2, 0)) 
plt.imshow(img.astype(np.float32))

NameError: name 'train_loader' is not defined

In [4]:
class CNN(torch.nn.Module): 
    def __init__(self, num_classes=3): 
        super().__init__() 
        self.model = torch.nn.Sequential( 
            torch.nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 3, padding = 1),  
            torch.nn.ReLU(), 
            torch.nn.MaxPool2d(kernel_size=2), 
  
            torch.nn.Conv2d(in_channels = 6, out_channels = 6, kernel_size = 3, padding = 1), 
            torch.nn.ReLU(), 
            torch.nn.MaxPool2d(kernel_size=2), 
              
            torch.nn.Conv2d(in_channels = 6, out_channels = 6, kernel_size = 3, padding = 1), 
            torch.nn.ReLU(), 
            torch.nn.MaxPool2d(kernel_size=2), 
  
            torch.nn.Flatten(), 
            torch.nn.Linear(4704, 512), 
            torch.nn.ReLU(), 
            torch.nn.Linear(512, num_classes) 
        ) 
  
    def forward(self, x): 
        return self.model(x) 

NameError: name 'torch' is not defined

In [2]:
#гиперпараметры
num_epochs = 2
learning_rate = 0.001
weight_decay = 0.01

net = CNN().to(device)

lossFn = nn.CrossEntropyLoss()
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
  

train_loss_list = [] 
import time
t = time.time()
for epoch in range(num_epochs): 
    print(f'Epoch {epoch+1}/{num_epochs}:', end = ' ') 
    train_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = net(images)
        
        loss = lossFn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        if i%10==0:
            print('Эпоха ' + str(epoch) + ' из ' + str(num_epochs) + ' Шаг ' +
                  str(i) + ' Ошибка: ', loss.item())
        
print('Time:',time.time() - t)


NameError: name 'CNN' is not defined

In [3]:
correct_predictions = 0
num_test_samples = len(test_sliced)

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        pred = net(images) 
        _, pred_class = torch.max(pred.data, 1) 
        correct_predictions += (pred_class == labels).sum().item()

print('Точность модели: ' + str(100 * correct_predictions / num_test_samples) + '%')

NameError: name 'test_sliced' is not defined

In [4]:
inputs, classes = next(iter(test_loader))
pred = net(inputs.to(device))
_, pred_class = torch.max(pred.data, 1)


for i,j in zip(inputs, pred_class):
    img = i.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.title(class_names[j])
    plt.pause(2)


NameError: name 'test_loader' is not defined

In [5]:

torch.save(net.state_dict(), 'net.ckpt')


NameError: name 'torch' is not defined